### Imports

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
import os
import math

In [3]:
import pandas as pd
import numpy as np

In [4]:
from pandas.api.types import is_string_dtype, is_numeric_dtype

In [5]:
pd.__version__

'0.23.4'

In [88]:
from sklearn.ensemble import RandomForestRegressor
from IPython.display import display

from sklearn import metrics
from sklearn.model_selection import TimeSeriesSplit

In [7]:
import sys
sys.path.append('../src')

In [8]:
from workflow.data_utils import load_csv

In [9]:
from workflow.structured import add_datepart, train_cats, proc_df, apply_cats

### Constants

In [10]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [11]:
def half_bool_columns_to_float_type(df):
    for col in df.columns:
        if is_string_dtype(df[col]):
            unqs = df[col].unique()
            if len(unqs) == 2 and (False in unqs or True in unqs):
                print(col)
                df[col] = df[col].astype(float)

In [12]:
# some duplicated info
drop_columns = ['date', 'sessionId', 'visitId']

### Loads 

In [13]:
%%time
full_train = load_csv('../data/processed_data/processed_train.csv',
                      low_memory=False, nrows=None)

CPU times: user 6.85 s, sys: 641 ms, total: 7.49 s
Wall time: 7.49 s


In [14]:
full_train.drop(columns=drop_columns, inplace=True)

In [15]:
half_bool_columns_to_float_type(full_train)

trafficSource_adwordsClickInfo.isVideoAd
trafficSource_isTrueDirect


In [16]:
full_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Data columns (total 32 columns):
channelGrouping                                 903653 non-null object
fullVisitorId                                   903653 non-null object
visitNumber                                     903653 non-null int64
visitStartTime                                  903653 non-null int64
device_browser                                  903653 non-null object
device_deviceCategory                           903653 non-null object
device_isMobile                                 903653 non-null bool
device_operatingSystem                          903653 non-null object
geoNetwork_city                                 903653 non-null object
geoNetwork_continent                            903653 non-null object
geoNetwork_country                              903653 non-null object
geoNetwork_metro                                903653 non-null object
geoNetwork_networkDomain                 

In [17]:
display_all(full_train.head())

,channelGrouping,fullVisitorId,visitNumber,visitStartTime,device_browser,device_deviceCategory,device_isMobile,device_operatingSystem,geoNetwork_city,geoNetwork_continent,geoNetwork_country,geoNetwork_metro,geoNetwork_networkDomain,geoNetwork_region,geoNetwork_subContinent,totals_bounces,totals_hits,totals_newVisits,totals_pageviews,trafficSource_adContent,trafficSource_adwordsClickInfo.adNetworkType,trafficSource_adwordsClickInfo.gclId,trafficSource_adwordsClickInfo.isVideoAd,trafficSource_adwordsClickInfo.page,trafficSource_adwordsClickInfo.slot,trafficSource_campaign,trafficSource_isTrueDirect,trafficSource_keyword,trafficSource_medium,trafficSource_referralPath,trafficSource_source,totals_transactionRevenue
0,Organic Search,1131660440785968503,1,1472830385,Chrome,desktop,False,Windows,Izmir,Asia,Turkey,(not set),ttnet.com.tr,Izmir,Western Asia,1.0,1,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google,NaN
1,Organic Search,377306020877927890,1,1472880147,Firefox,desktop,False,Macintosh,not available in demo dataset,Oceania,Australia,not available in demo dataset,dodo.net.au,not available in demo dataset,Australasia,1.0,1,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google,NaN
2,Organic Search,3895546263509774583,1,1472865386,Chrome,desktop,False,Windows,Madrid,Europe,Spain,(not set),unknown.unknown,Community of Madrid,Southern Europe,1.0,1,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google,NaN
3,Organic Search,4763447161404445595,1,1472881213,UC Browser,desktop,False,Linux,not available in demo dataset,Asia,Indonesia,not available in demo dataset,unknown.unknown,not available in demo dataset,Southeast Asia,1.0,1,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,(not set),NaN,google + online,organic,NaN,google,NaN
4,Organic Search,27294437909732085,2,1472822600,Chrome,mobile,True,Android,not available in demo dataset,Europe,United Kingdom,not available in demo dataset,unknown.unknown,not available in demo dataset,Northern Europe,1.0,1,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,(not set),1.0,(not provided),organic,NaN,google,NaN


In [18]:
%%time
test_df = load_csv('../data/processed_data/processed_test.csv', low_memory=False, nrows=None)

CPU times: user 6.23 s, sys: 456 ms, total: 6.68 s
Wall time: 6.68 s


In [19]:
test_df.drop(columns=drop_columns, inplace=True)

In [20]:
half_bool_columns_to_float_type(test_df)

trafficSource_adwordsClickInfo.isVideoAd
trafficSource_isTrueDirect


In [21]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 804684 entries, 0 to 804683
Data columns (total 31 columns):
channelGrouping                                 804684 non-null object
fullVisitorId                                   804684 non-null object
visitNumber                                     804684 non-null int64
visitStartTime                                  804684 non-null int64
device_browser                                  804684 non-null object
device_deviceCategory                           804684 non-null object
device_isMobile                                 804684 non-null bool
device_operatingSystem                          804684 non-null object
geoNetwork_city                                 804684 non-null object
geoNetwork_continent                            804684 non-null object
geoNetwork_country                              804684 non-null object
geoNetwork_metro                                804684 non-null object
geoNetwork_networkDomain                 

#### Check column names and types are == for train and test

In [34]:
all(full_train.drop('totals_transactionRevenue', axis=1).columns == test_df.columns)

True

In [35]:
types_equal = []
for col_name in test_df.columns:
    types_equal.append(full_train[col_name].dtype == test_df[col_name].dtype)

all(types_equal)

True

In [53]:
display_all(full_train.describe(include='all').T)

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
channelGrouping,10000,7,Organic Search,4730,NaN,NaN,NaN,NaN,NaN,NaN,NaN
visitNumber,10000,NaN,NaN,NaN,2.1673,7.86228,1,1,1,1,317
device_browser,10000,22,Chrome,6917,NaN,NaN,NaN,NaN,NaN,NaN,NaN
device_deviceCategory,10000,3,desktop,7115,NaN,NaN,NaN,NaN,NaN,NaN,NaN
device_isMobile,10000,2,False,7114,NaN,NaN,NaN,NaN,NaN,NaN,NaN
device_operatingSystem,10000,13,Windows,3977,NaN,NaN,NaN,NaN,NaN,NaN,NaN
geoNetwork_city,10000,109,not available in demo dataset,5941,NaN,NaN,NaN,NaN,NaN,NaN,NaN
geoNetwork_continent,10000,6,Americas,5235,NaN,NaN,NaN,NaN,NaN,NaN,NaN
geoNetwork_country,10000,138,United States,4274,NaN,NaN,NaN,NaN,NaN,NaN,NaN
geoNetwork_metro,10000,22,not available in demo dataset,5941,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Workflow

In [25]:
full_train['totals_transactionRevenue'] = full_train['totals_transactionRevenue'].fillna(value=0)

In [26]:
# posix seconds to datetime
full_train.visitStartTime = pd.to_datetime(full_train.visitStartTime, unit='s')

In [27]:
test_df.visitStartTime = pd.to_datetime(test_df.visitStartTime, unit='s')

#### Take fullVisitorId -> validation ids, if it has transaction after 2017-04-30 

In [28]:
validationIds = full_train[full_train.visitStartTime > '2017-04-30'].fullVisitorId.unique()

In [29]:
validationIds.shape

(161118,)

In [30]:
# save for later use
np.save('../data/tmp/validIds.npy',validationIds)

#### Continue workflow

In [31]:
# make time features
add_datepart(full_train, 'visitStartTime', time=False)

In [32]:
# make time features
add_datepart(test_df, 'visitStartTime', time=False)

In [33]:
set(full_train.drop('totals_transactionRevenue', axis=1).columns) - set(test_df.columns)

set()

#### Drop fullVisitorId columns to not categorize them

In [36]:
train_visitorIds = full_train.fullVisitorId
test_visitorIds = test_df.fullVisitorId

In [37]:
full_train.drop(columns=['fullVisitorId'], inplace=True)
test_df.drop(columns=['fullVisitorId'], inplace=True)

In [38]:
# make categorical data
train_cats(full_train)

In [39]:
apply_cats(test_df, full_train)

In [40]:
display_all(full_train.isnull().sum().sort_index()/len(full_train))

channelGrouping                                 0.000000
device_browser                                  0.000000
device_deviceCategory                           0.000000
device_isMobile                                 0.000000
device_operatingSystem                          0.000000
geoNetwork_city                                 0.000000
geoNetwork_continent                            0.000000
geoNetwork_country                              0.000000
geoNetwork_metro                                0.000000
geoNetwork_networkDomain                        0.000000
geoNetwork_region                               0.000000
geoNetwork_subContinent                         0.000000
totals_bounces                                  0.501324
totals_hits                                     0.000000
totals_newVisits                                0.221980
totals_pageviews                                0.000111
totals_transactionRevenue                       0.000000
trafficSource_adContent        

In [41]:
display_all(test_df.isnull().sum().sort_index()/len(test_df))

channelGrouping                                 0.000000
device_browser                                  0.019764
device_deviceCategory                           0.000000
device_isMobile                                 0.000000
device_operatingSystem                          0.001150
geoNetwork_city                                 0.009569
geoNetwork_continent                            0.000000
geoNetwork_country                              0.000034
geoNetwork_metro                                0.001099
geoNetwork_networkDomain                        0.031993
geoNetwork_region                               0.007047
geoNetwork_subContinent                         0.000000
totals_bounces                                  0.476878
totals_hits                                     0.000000
totals_newVisits                                0.248935
totals_pageviews                                0.000173
trafficSource_adContent                         0.994974
trafficSource_adwordsClickInfo.

In [42]:
# add back fullVisitosIds
full_train['fullVisitorId'] = train_visitorIds
test_df['fullVisitorId'] = test_visitorIds

In [43]:
%%time
os.makedirs('../data/tmp', exist_ok=True)
full_train.to_feather('../data/tmp/raw_train')

CPU times: user 373 ms, sys: 183 ms, total: 556 ms
Wall time: 439 ms


In [44]:
%%time
os.makedirs('../data/tmp', exist_ok=True)
test_df.to_feather('../data/tmp/raw_test')

CPU times: user 215 ms, sys: 146 ms, total: 361 ms
Wall time: 499 ms


#### Preprocess

In [40]:
full_train = pd.read_feather('../data/tmp/raw_train')

/home/ademyanchuk/.virtualenvs/kaggle_env/lib/python3.6/site-packages/pandas/io/feather_format.py:112: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  return feather.read_dataframe(path, nthreads=nthreads)


In [45]:
%%time
df, y, nas = proc_df(full_train, 'totals_transactionRevenue', skip_flds=['fullVisitorId'], max_n_cat=100)

CPU times: user 2.19 s, sys: 929 ms, total: 3.12 s
Wall time: 3.13 s


In [46]:
df.columns.shape

(321,)

In [47]:
display_all(df.head(10))

,visitNumber,device_isMobile,geoNetwork_city,geoNetwork_country,geoNetwork_networkDomain,geoNetwork_region,totals_bounces,totals_hits,totals_newVisits,totals_pageviews,trafficSource_adwordsClickInfo.gclId,trafficSource_adwordsClickInfo.isVideoAd,trafficSource_adwordsClickInfo.page,trafficSource_isTrueDirect,trafficSource_keyword,trafficSource_referralPath,trafficSource_source,visitStartTimeYear,visitStartTimeMonth,visitStartTimeWeek,visitStartTimeDay,visitStartTimeDayofweek,visitStartTimeDayofyear,visitStartTimeIs_month_end,visitStartTimeIs_month_start,visitStartTimeIs_quarter_end,visitStartTimeIs_quarter_start,visitStartTimeIs_year_end,visitStartTimeIs_year_start,visitStartTimeElapsed,totals_bounces_na,totals_newVisits_na,totals_pageviews_na,trafficSource_adwordsClickInfo.isVideoAd_na,trafficSource_adwordsClickInfo.page_na,trafficSource_isTrueDirect_na,channelGrouping_(Other),channelGrouping_Affiliates,channelGrouping_Direct,channelGrouping_Display,channelGrouping_Organic Search,channelGrouping_Paid Search,channelGrouping_Referral,channelGrouping_Social,channelGrouping_nan,device_browser_(not set),device_browser_0,device_browser_ADM,device_browser_Amazon Silk,device_browser_Android Browser,device_browser_Android Runtime,device_browser_Android Webview,device_browser_Apple-iPhone7C2,device_browser_BlackBerry,device_browser_CSM Click,device_browser_Changa 99695759,device_browser_Chrome,device_browser_Coc Coc,device_browser_DASH_JR_3G,device_browser_DoCoMo,device_browser_Edge,device_browser_Firefox,device_browser_HTC802t_TD,device_browser_Hisense M20-M_LTE,device_browser_IE with Chrome Frame,device_browser_Internet Explorer,device_browser_Iron,device_browser_Konqueror,device_browser_LYF_LS_4002_11,device_browser_LYF_LS_4002_12,device_browser_Lunascape,device_browser_M5,device_browser_MQQBrowser,device_browser_MRCHROME,device_browser_Maxthon,device_browser_Mozilla,device_browser_Mozilla Compatible Agent,device_browser_Nichrome,device_browser_Nintendo Browser,device_browser_Nokia Browser,device_browser_NokiaE52-1,device_browser_Opera,device_browser_Opera Mini,device_browser_Puffin,device_browser_Reddit,device_browser_Safari,device_browser_Safari (in-app),device_browser_SeaMonkey,device_browser_Seznam,device_browser_TCL P500M,device_browser_ThumbSniper,device_browser_UC Browser,device_browser_User Agent,device_browser_YE,device_browser_YaBrowser,device_browser_[Use default User-agent string] LIVRENPOCHE,device_browser_no-ua,device_browser_osee2unifiedRelease,device_browser_subjectAgent: NoticiasBoom,device_browser_nan,device_deviceCategory_desktop,device_deviceCategory_mobile,device_deviceCategory_tablet,device_deviceCategory_nan,device_operatingSystem_(not set),device_operatingSystem_Android,device_operatingSystem_BlackBerry,device_operatingSystem_Chrome OS,device_operatingSystem_Firefox OS,device_operatingSystem_FreeBSD,device_operatingSystem_Linux,device_operatingSystem_Macintosh,device_operatingSystem_NTT DoCoMo,device_operatingSystem_Nintendo 3DS,device_operatingSystem_Nintendo Wii,device_operatingSystem_Nintendo WiiU,device_operatingSystem_Nokia,device_operatingSystem_OpenBSD,device_operatingSystem_Samsung,device_operatingSystem_SunOS,device_operatingSystem_Windows,device_operatingSystem_Windows Phone,device_operatingSystem_Xbox,device_operatingSystem_iOS,device_operatingSystem_nan,geoNetwork_continent_(not set),geoNetwork_continent_Africa,geoNetwork_continent_Americas,geoNetwork_continent_Asia,geoNetwork_continent_Europe,geoNetwork_continent_Oceania,geoNetwork_continent_nan,geoNetwork_metro_(not set),geoNetwork_metro_Abilene-Sweetwater TX,geoNetwork_metro_Albany-Schenectady-Troy NY,geoNetwork_metro_Atlanta GA,geoNetwork_metro_Augusta GA,geoNetwork_metro_Austin TX,geoNetwork_metro_Baltimore MD,geoNetwork_metro_Boise ID,geoNetwork_metro_Boston MA-Manchester NH,geoNetwork_metro_Butte-Bozeman MT,geoNetwork_metro_Central Scotland,geoNetwork_metro_Charleston SC,geoNetwork_metro_Charlotte NC,geoNetwork_metro_Charlottesville VA,geoNe

In [48]:
test, _, _ = proc_df(test_df, skip_flds=['fullVisitorId'], max_n_cat=100, na_dict=nas)

In [49]:
test.columns.shape

(321,)

In [50]:
display_all(test.head())

,visitNumber,device_isMobile,geoNetwork_city,geoNetwork_country,geoNetwork_networkDomain,geoNetwork_region,totals_bounces,totals_hits,totals_newVisits,totals_pageviews,trafficSource_adwordsClickInfo.gclId,trafficSource_adwordsClickInfo.isVideoAd,trafficSource_adwordsClickInfo.page,trafficSource_isTrueDirect,trafficSource_keyword,trafficSource_referralPath,trafficSource_source,visitStartTimeYear,visitStartTimeMonth,visitStartTimeWeek,visitStartTimeDay,visitStartTimeDayofweek,visitStartTimeDayofyear,visitStartTimeIs_month_end,visitStartTimeIs_month_start,visitStartTimeIs_quarter_end,visitStartTimeIs_quarter_start,visitStartTimeIs_year_end,visitStartTimeIs_year_start,visitStartTimeElapsed,totals_bounces_na,totals_newVisits_na,totals_pageviews_na,trafficSource_adwordsClickInfo.isVideoAd_na,trafficSource_adwordsClickInfo.page_na,trafficSource_isTrueDirect_na,channelGrouping_(Other),channelGrouping_Affiliates,channelGrouping_Direct,channelGrouping_Display,channelGrouping_Organic Search,channelGrouping_Paid Search,channelGrouping_Referral,channelGrouping_Social,channelGrouping_nan,device_browser_(not set),device_browser_0,device_browser_ADM,device_browser_Amazon Silk,device_browser_Android Browser,device_browser_Android Runtime,device_browser_Android Webview,device_browser_Apple-iPhone7C2,device_browser_BlackBerry,device_browser_CSM Click,device_browser_Changa 99695759,device_browser_Chrome,device_browser_Coc Coc,device_browser_DASH_JR_3G,device_browser_DoCoMo,device_browser_Edge,device_browser_Firefox,device_browser_HTC802t_TD,device_browser_Hisense M20-M_LTE,device_browser_IE with Chrome Frame,device_browser_Internet Explorer,device_browser_Iron,device_browser_Konqueror,device_browser_LYF_LS_4002_11,device_browser_LYF_LS_4002_12,device_browser_Lunascape,device_browser_M5,device_browser_MQQBrowser,device_browser_MRCHROME,device_browser_Maxthon,device_browser_Mozilla,device_browser_Mozilla Compatible Agent,device_browser_Nichrome,device_browser_Nintendo Browser,device_browser_Nokia Browser,device_browser_NokiaE52-1,device_browser_Opera,device_browser_Opera Mini,device_browser_Puffin,device_browser_Reddit,device_browser_Safari,device_browser_Safari (in-app),device_browser_SeaMonkey,device_browser_Seznam,device_browser_TCL P500M,device_browser_ThumbSniper,device_browser_UC Browser,device_browser_User Agent,device_browser_YE,device_browser_YaBrowser,device_browser_[Use default User-agent string] LIVRENPOCHE,device_browser_no-ua,device_browser_osee2unifiedRelease,device_browser_subjectAgent: NoticiasBoom,device_browser_nan,device_deviceCategory_desktop,device_deviceCategory_mobile,device_deviceCategory_tablet,device_deviceCategory_nan,device_operatingSystem_(not set),device_operatingSystem_Android,device_operatingSystem_BlackBerry,device_operatingSystem_Chrome OS,device_operatingSystem_Firefox OS,device_operatingSystem_FreeBSD,device_operatingSystem_Linux,device_operatingSystem_Macintosh,device_operatingSystem_NTT DoCoMo,device_operatingSystem_Nintendo 3DS,device_operatingSystem_Nintendo Wii,device_operatingSystem_Nintendo WiiU,device_operatingSystem_Nokia,device_operatingSystem_OpenBSD,device_operatingSystem_Samsung,device_operatingSystem_SunOS,device_operatingSystem_Windows,device_operatingSystem_Windows Phone,device_operatingSystem_Xbox,device_operatingSystem_iOS,device_operatingSystem_nan,geoNetwork_continent_(not set),geoNetwork_continent_Africa,geoNetwork_continent_Americas,geoNetwork_continent_Asia,geoNetwork_continent_Europe,geoNetwork_continent_Oceania,geoNetwork_continent_nan,geoNetwork_metro_(not set),geoNetwork_metro_Abilene-Sweetwater TX,geoNetwork_metro_Albany-Schenectady-Troy NY,geoNetwork_metro_Atlanta GA,geoNetwork_metro_Augusta GA,geoNetwork_metro_Austin TX,geoNetwork_metro_Baltimore MD,geoNetwork_metro_Boise ID,geoNetwork_metro_Boston MA-Manchester NH,geoNetwork_metro_Butte-Bozeman MT,geoNetwork_metro_Central Scotland,geoNetwork_metro_Charleston SC,geoNetwork_metro_Charlotte NC,geoNetwork_metro_Charlottesville VA,geoNe

In [51]:
all(df.columns == test.columns)

True

In [52]:
y.mean()

1704272.8126836296

In [53]:
y = np.log1p(y)

In [54]:
m = RandomForestRegressor(n_jobs=-1, n_estimators=100)
m.fit(df, y)
m.score(df,y)

0.9029624185260904

#### Train valid split

In [55]:
all(full_train.index == df.index)

True

In [56]:
valid_index = full_train[full_train.fullVisitorId.isin(validationIds)].index

In [57]:
train_index = full_train[~full_train.fullVisitorId.isin(validationIds)].index

In [58]:
def split_vals_array(a, train_index, val_index):
    return a[train_index].copy(), a[val_index].copy()

def split_vals_df(a, train_index, val_index):
    return a.iloc[train_index].copy(), a.iloc[val_index].copy()


raw_train, raw_valid = split_vals_df(full_train, train_index, valid_index)
X_train, X_valid = split_vals_df(df, train_index, valid_index)
y_train, y_valid = split_vals_array(y, train_index, valid_index)

X_train.shape, y_train.shape, X_valid.shape

((681849, 321), (681849,), (221804, 321))

In [59]:
y_train.mean(), y_valid.mean()

(0.20146147902692166, 0.3059895624328279)

In [60]:
y_train.max(), y_valid.max()

(23.497337833653027, 23.86437469605166)

#### Example of TimeSeriesSplit

In [89]:
X = np.array([[1, 2], [3, 4], [1, 2], [3, 4], [1, 2], [3, 4]])
y = np.array([1, 2, 3, 4, 5, 6])
tscv = TimeSeriesSplit(n_splits=5)
print(tscv)  

for train_index, test_index in tscv.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

TimeSeriesSplit(max_train_size=None, n_splits=5)
TRAIN: [0] TEST: [1]
TRAIN: [0 1] TEST: [2]
TRAIN: [0 1 2] TEST: [3]
TRAIN: [0 1 2 3] TEST: [4]
TRAIN: [0 1 2 3 4] TEST: [5]


In [90]:
tscv = TimeSeriesSplit(n_splits=5)

#### Base Model 

In [61]:
def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
           m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'):
        res.append(m.oob_score_)
    print(res)

In [62]:
def rmse(x, y):
    return math.sqrt(((x-y)**2).mean())

In [63]:
m_val = RandomForestRegressor(n_jobs=-1, n_estimators=100)
%time m_val.fit(X_train, y_train)

CPU times: user 46min 33s, sys: 50.6 s, total: 47min 24s
Wall time: 6min 57s


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

#### Visit Level

In [64]:
print_score(m_val)

[0.5976430330813959, 2.1541521568709947, 0.8997188839647949, 0.14058583153614346]


#### Go to User Level

In [65]:
val_preds = m_val.predict(X_valid)

In [66]:
val_preds[val_preds < 0] = 0.

In [67]:
grouped = raw_valid[['fullVisitorId']].copy()

In [68]:
grouped['y'] = np.expm1(y_valid)
grouped['y_hat'] = np.expm1(val_preds)

In [69]:
grouped = grouped.groupby('fullVisitorId', as_index=False).sum()

In [70]:
grouped['y'] = np.log1p(grouped['y'])
grouped['y_hat'] = np.log1p(grouped['y_hat'])

In [71]:
rmse(grouped.y, grouped.y_hat)

2.1447894992588807

####  Test flow

In [72]:
test_preds = m.predict(test)
test_preds[test_preds < 0] = 0.

In [76]:
test_grouped = test_df[['fullVisitorId']].copy()
test_grouped['y_hat'] = np.expm1(test_preds)

In [79]:
test_grouped = test_grouped.groupby('fullVisitorId', as_index=False).sum()
test_grouped['y_hat'] = np.log1p(test_grouped['y_hat'])

In [82]:
submit = pd.read_csv('../data/sample_submission.csv')

In [83]:
test_grouped.fullVisitorId.tolist() == submit.fullVisitorId.tolist()

True

In [84]:
submit['fullVisitorId'] = test_grouped['fullVisitorId']

In [85]:
submit['PredictedLogRevenue'] = test_grouped['y_hat']

In [86]:
os.makedirs('../submissions', exist_ok=True)

In [87]:
submit.to_csv('../submissions/rf_baseline.csv', header=True, index=False)